In [7]:
import pandas as pd
import requests
import os

# Path to the training dataset
train_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', '04_feature', 'second_features_selection.csv'))

# Path to the production dataset
prod_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', '04_feature', 'test_dataset_features.csv'))

# Load the datasets
train_data = pd.read_csv(train_data_path)
prod_data = pd.read_csv(prod_data_path)


In [8]:
raw_body = prod_data.to_dict(orient='records')  # Send the first 5 rows as an example
body = [{k: (None if pd.isna(v) else v) for k, v in row.items()} for row in raw_body]

In [9]:
url = "http://127.0.0.1:5000/predict"

response = requests.post(url, json=body)
print(response.json())

{'scores': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [11]:
prod_data['TARGET'] = response.json()['scores']

In [16]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# Define column mapping if necessary
column_mapping = ColumnMapping(
    target='TARGET',  # Replace with your target column name if applicable
    prediction=None,  # Replace with your prediction column name if applicable
    numerical_features=train_data.columns.tolist(),  # Replace with your numerical feature columns
    categorical_features=[]  # Replace with your categorical feature columns
)

# Create the data drift report
report = Report(metrics=[
    DataDriftPreset(),
])

report.run(reference_data=train_data, current_data=prod_data, column_mapping=column_mapping)

# Save the report as an HTML file
report_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', '08_reporting', 'data_drift_report.html'))
report.save_html(report_path)


In [15]:
import webbrowser

# Open the HTML report in the default web browser
webbrowser.open(report_path)


True